In [1]:
cd /home/tvangraft/tudelft/thesis/metaengineering

/home/tvangraft/tudelft/thesis/metaengineering


In [2]:
from collections import defaultdict
from typing import DefaultDict, List, Hashable, Dict, Any

from src.utils.utils import get_generator, get_project_root
from src.utils.test_result_store import TestResultStore

from src.pipeline.config import DataLoaderConfig, TaskLoaderConfig
from src.pipeline.taskloader import TaskLoader, TaskFrame
from src.pipeline.dataloader import DataLoader

from src.orchestrator.trainer import Trainer

from src.settings.tier import Tier
from src.settings.strategy import Strategy
from src.settings.metabolites import ENZYMES, METABOLITES, PRECURSOR_METABOLITES, PRECURSOR_METABOLITES_NO_TRANSFORM

from src.gnn.data_augmentation import DataAugmentation
from src.gnn.embeddings import generate_embedding
from src.gnn.graph_builder import get_samples_hetero_graph, get_graph_fc, get_graph_fc_protein_only, edge_index_from_df_protein_only, get_samples_graph
from src.gnn.shared_training import count_parameters, log_metrics, tune_metabolite_hyper_parameters

import pandas as pd
import numpy as np

import cobra
from cobra.util import create_stoichiometric_matrix
from cobra.core import Reaction

import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from more_itertools import flatten

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_absolute_error

from scipy.stats import pearsonr

import os
from functools import partial

from tqdm import tqdm

import torch
from torch.nn import BatchNorm1d, ModuleList
import torch.nn.functional as F

from torch_geometric.data import Data, HeteroData
from torch_geometric.utils import from_networkx, to_networkx
from torch_geometric.loader import DataLoader as GeoDataLoader
from torch_geometric.nn import GAT, GCNConv, to_hetero, SAGEConv, GATConv, HeteroLinear, Linear, Node2Vec
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch_geometric.transforms as T
from torch_geometric.nn.conv import HeteroConv

import mlflow.pytorch

from config import HYPERPARAMETERS, BEST_PARAMETERS

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler, FIFOScheduler
from ray.air import session, RunConfig
from ray.tune.integration.mlflow import mlflow_mixin
from ray.tune.integration.mlflow import MLflowLoggerCallback

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mlflow.set_tracking_uri("http://localhost:5000")
device = torch.device("cpu")
torch.manual_seed(42)

/home/tvangraft/tudelft/thesis/.env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = "/home/tvangraft/tudelft/thesis/metaengineering/data"
model = cobra.io.read_sbml_model(f'{path}/iMM904.xml')

In [4]:
def prepare_data(path: str, valid_metabolites: List[str]):
    edge_list_df = pd.read_csv(path)
    graph_fc = get_graph_fc_protein_only(edge_list_df, valid_metabolites)
    edge_index = edge_index_from_df_protein_only(graph_fc, edge_list_df, valid_metabolites)
    embedding = generate_embedding(edge_index.T, device, hetero=False)

    return edge_list_df, graph_fc, edge_index, embedding

# Model

In [19]:
class GATModel(torch.nn.Module):
    def __init__(self, model_config) -> None:
        super(GATModel, self).__init__()
        embedding_size = model_config["model_embedding_size"]
        n_heads = model_config["model_attention_heads"]
        self.n_layers = model_config["model_layers"]
        
        self.conv_layers = ModuleList([])
        self.transf_layers = ModuleList([])
        self.pooling_layers = ModuleList([])
        self.bn_layers = ModuleList([])
        
        self.conv1 = GATConv(
            -1, out_channels=embedding_size, heads=n_heads, add_self_loops=False, bias=False
        )
        self.transf1 = Linear(
            in_channels=embedding_size*n_heads, 
            out_channels=embedding_size, 
        )

        for i in range(self.n_layers):
            self.conv_layers.append(
                GATConv(
                    -1, 
                    out_channels=embedding_size, 
                    heads=n_heads, 
                    add_self_loops=False
                )
            )
            self.transf_layers.append(
                Linear(
                    embedding_size*n_heads, 
                    embedding_size
                )
            )

        self.linear1 = Linear(embedding_size * 2, embedding_size)
        self.linear2 = Linear(embedding_size, model_config['output_size'])
    
    def forward(self, x, edge_index, batch_index):
        x = self.conv1(x, edge_index)

        global_representation = []

        for i in range(self.n_layers):
            x = self.conv_layers[i](x, edge_index)
            print(x.shape)
            x = torch.relu(self.transf_layers[i](x))
            print(x.shape)
            global_representation.append(torch.cat([gmp(x, batch_index), gap(x, batch_index)], dim=1))

        # This generates the last embeddings for all enzymes in the graph
        x = sum(global_representation)
        x = torch.relu(self.linear1(x))
        x = F.dropout(x, p=0.8, training=self.training)
        x = self.linear2(x)
        return x 

# Setup functions

In [61]:
def train_one_epoch(epoch, model, train_loader, optimizer, loss_fn, debug=False):
    # Enumerate over the data
    running_loss = 0.0
    step = 0
    for _, batch in enumerate(train_loader):
        print(batch)
        # Use GPU
        batch.to(device)
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        pred = model.forward(
            batch.x, 
            batch.edge_index,
            batch.batch
        )
        # Calculating the loss and gradients
        train_mask = torch.reshape(batch.train_mask.bool(), pred.shape)

        if debug:
            print(f"{pred.shape=}")
            print(f"{train_mask.sum()=}")
            print(
                f"{pred.shape=} \n"
                f"{pred[train_mask].mean()=} \n"
                f"{pred[train_mask].max()=} \n"
                f"{pred[train_mask].min()=} \n"
                f"{pred[train_mask].shape=} \n"
            )
            print(
                f"{torch.squeeze(pred).shape} \n"
                f"{torch.squeeze(batch.y.float()).shape} \n"
            )

        loss = loss_fn(
            torch.squeeze(pred), 
            torch.squeeze(torch.reshape(batch.y.float(), pred.shape))
        )

        loss.backward()  
        optimizer.step()  
        # Update tracking
        running_loss += loss.item()
        step += 1
    
    tune.report(loss=(running_loss/step))

    return running_loss/step
    
@mlflow_mixin
def test(epoch, model, test_loader, loss_fn, debug=False):
    all_preds_raw = []
    all_labels = []
    all_knockout_ids = []
    running_loss = 0.0
    step = 0
    for batch in test_loader:
        batch.to(device)
        # print(batch)  
        pred = model(
            batch.x, 
            batch.edge_index,
            batch.batch,
        )
        test_mask = torch.reshape(batch.test_mask.bool(), pred.shape)

        if debug:
            print(f"{test_mask.sum()=}")
            print(
                f"{pred.shape=} \n"
                f"{pred[test_mask].mean()=} \n"
                f"{pred[test_mask].max()=} \n"
                f"{pred[test_mask].min()=} \n"
                f"{pred[test_mask].shape=} \n"
            ) 
        
        loss = loss_fn(
            torch.squeeze(torch.nan_to_num(pred[test_mask])), 
            torch.squeeze(torch.nan_to_num(batch.y.float()[torch.squeeze(test_mask)]))
        )

         # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds_raw.append(torch.nan_to_num(pred[test_mask]).cpu().detach().numpy())
        all_labels.append(torch.nan_to_num(batch.y[torch.squeeze(test_mask)]).cpu().detach().numpy())
        # all_knockout_ids.append(batch['enzymes'].knockout_label_id.cpu().detach().numpy())
    
    all_preds_raw = np.concatenate(all_preds_raw).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    # all_knockout_ids = np.concatenate(all_knockout_ids).ravel()
    log_metrics(all_preds_raw, all_labels, all_knockout_ids, epoch, "test")
    return running_loss/step

def build_model(model_config, output_size):
    print(f"creating model {model_config=}")
    # Set the number of outputs we expect
    model_config['output_size'] = output_size
    params = model_config

    if 'mlflow' in model_config:
        run_id = model_config['mlflow']['tags']['mlflow.parentRunId']
        mlflow.set_tag("mlflow.parentRunId", run_id)
    # Logging params
    for key in params.keys():
        mlflow.log_param(key, params[key])
  
    # Loading the model
    print("Loading model...")
    model = GATModel(model_config=params)
    model = model.to(device)

    return model

@mlflow_mixin
def run_one_training(model_config, train_samples, test_samples, checkpoint_dir):
    # Build the model
    model = build_model(model_config, train_samples[0].y.shape[0])

    # Preparing training
    train_loader = GeoDataLoader(train_samples, batch_size=model_config['batch_size'])
    test_loader = GeoDataLoader(test_samples, batch_size=1)

    # < 1 increases precision, > 1 recall
    # loss_fn = torch.nn.MSELoss(reduction='none')
    loss_fn = torch.nn.MSELoss()
    # we need to keep the lr quite low since otherwise the weights explode
    optimizer = torch.optim.SGD(
        model.parameters(), 
        lr=model_config['learning_rate'],
        momentum=model_config['sgd_momentum'],
        # weight_decay=5e-4
    )
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=model_config['scheduler_gamma'])

    # Use this to debug the train and test function
    debug = False
    
    # Start training
    best_loss = 1000
    early_stopping_counter = 0
    max_epochs = 300
    for epoch in tqdm(range(max_epochs)): 
        if early_stopping_counter <= 25: # = x * 5 
            # Training
            model.train()
            loss = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn, debug=debug)
            # print(f"Epoch {epoch} | Train Loss {loss}")
            mlflow.log_metric(key="Train loss", value=float(loss), step=epoch)
            
            # Testing
            model.eval()
            if epoch % 5 == 0 or epoch == max_epochs - 1:
                loss = test(epoch, model, test_loader, loss_fn, debug=debug)
                # print(f"Epoch {epoch} | Test Loss {loss}")
                mlflow.log_metric(key="Test loss", value=float(loss), step=epoch)
                
                # Update best loss
                if float(loss) < best_loss:
                    best_loss = loss
                    # Save the currently best model 
                    # mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1

            scheduler.step()
            mlflow.log_metric(key="Learning rate", value=float(scheduler.get_last_lr()[0]), step=epoch)
            
        else:
            print("Early stopping due to no improvement.")
            session.report({
                "loss": best_loss
            })
            return {"loss": best_loss}
    print(f"Finishing training with best test loss: {best_loss}")

    with torch.no_grad():
        sample = train_samples[0].to(device)
        model.forward(sample.x_dict, sample.edge_index_dict)
        print(f"Number of parameters: {count_parameters(model)}")

    session.report({
        "loss": best_loss
    })

    mlflow.end_run()

    return {"loss": best_loss}

In [7]:
HYPERPARAMETERS = {
    "batch_size": tune.choice([2, 4, 8]),
    "learning_rate": tune.choice([0.1, 0.05, 0.01, 0.001]),
    "sgd_momentum": tune.choice([0.9, 0.8, 0.5]),
    "scheduler_gamma": tune.choice([0.995, 1]),
    "model_embedding_size": tune.choice([8, 16, 32, 64, 128]),
    "model_attention_heads": tune.choice([1, 2, 3, 4]),
    "model_layers": tune.choice([1, 3, 5, 7]),
}

In [ ]:
edge_list_df_unfiltered, graph_fc_unfiltered, edge_index_unfiltered, embedding_unfiltered = prepare_data(
    './data/training/edge_list_unfiltered_protein_only.csv',
    PRECURSOR_METABOLITES_NO_TRANSFORM,
)
edge_list_df_strict, graph_fc_strict, edge_index_strict, embedding_strict = prepare_data(
    './data/training/edge_list_strict_protein_only.csv',
    PRECURSOR_METABOLITES,
)
edge_list_df_all, graph_fc_all, edge_index_all, embedding_all = prepare_data(
    './data/training/edge_list_all_protein_only.csv',
    METABOLITES,
)

In [41]:
def get_train_test_split(mode, metabolite_id, strategy):
    if mode == 'unfiltered':
        train_samples, test_samples = get_samples_graph(
            metabolite_id, strategy, PRECURSOR_METABOLITES_NO_TRANSFORM, graph_fc_unfiltered, edge_index_unfiltered, embedding_unfiltered
        )
    elif mode == 'strict':
        train_samples, test_samples = get_samples_graph(
            target_metabolite_id=metabolite_id, strategy=strategy, valid_metabolites=PRECURSOR_METABOLITES, graph_fc_df=graph_fc_strict, edge_index=edge_index_strict, node_embeddings=embedding_strict,
        )
    elif mode == 'all':
        train_samples, test_samples = get_samples_graph(
            metabolite_id, strategy, METABOLITES, graph_fc_all, edge_index_all, embedding_all
        )
    return train_samples, test_samples

In [ ]:
mlflow.end_run()
train_samples, test_samples = get_train_test_split('unfiltered', 'pyr', Strategy.ALL)
print(test_samples[0])

model_config = {
    "batch_size": 6,
    "learning_rate": 0.001,
    "sgd_momentum": 0.8,
    "scheduler_gamma": 1,
    "model_embedding_size": 16,
    "model_attention_heads": 1,
    "model_layers": 1,
}

run_one_training(model_config, train_samples, test_samples, None)
mlflow.end_run()

# Training

In [62]:
mlflow.end_run()
for strategy in [Strategy.ALL, Strategy.ONE_VS_ALL, Strategy.METABOLITE_CENTRIC]:
    for mode, graph_fc in [('unfiltered', graph_fc_unfiltered), ('strict', graph_fc_strict), ('all', graph_fc_all)]:
    # for mode, graph_fc in [('unfiltered', graph_fc_unfiltered)]:
        run_name = "model_gat_node_embeddings"
        experiment_name = f'protein_only_sweep_{mode}'
        mlflow.set_experiment(experiment_name)
        with mlflow.start_run(run_name=run_name) as run:
            for metabolite_id in list(set(graph_fc.columns.difference(ENZYMES).to_list()) & set(PRECURSOR_METABOLITES)):
                print(f"training {metabolite_id=}")
                train_samples, test_samples = get_train_test_split(mode, metabolite_id, strategy)
                print(len(train_samples))

                with mlflow.start_run(run_name=f"model_{metabolite_id}_{strategy}", nested=True):
                    result = tune_metabolite_hyper_parameters(experiment_name, HYPERPARAMETERS, train_samples, test_samples, run_fc=run_one_training, num_samples=16)
                    print(result)

== Status ==
Current time: 2022-12-19 12:24:24 (running for 00:00:22.32)
Memory usage on this node: 8.1/23.5 GiB 
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 160.000: None | Iter 80.000: None | Iter 40.000: None | Iter 20.000: -0.5825647979974746 | Iter 10.000: -0.5837701894342899
Resources requested: 13.0/16 CPUs, 0/0 GPUs, 0.0/12.27 GiB heap, 0.0/6.14 GiB objects
Result logdir: /home/tvangraft/ray_results/run_one_training_2022-12-19_12-24-01
Number of trials: 16/16 (13 RUNNING, 3 TERMINATED)
+------------------------------+------------+---------------------+--------------+-----------------+------------------------+------------------------+----------------+-------------------+----------------+------------+----------------------+
| Trial name                   | status     | loc                 |   batch_size |   learning_rate |   model_attention_head |   model_embedding_size |   model_layers |   scheduler_gamma |   sgd_momentum |       loss |   training_iteration |
|            

2022-12-19 12:26:58,707	INFO tune.py:762 -- Total run time: 176.79 seconds (176.66 seconds for the tuning loop).


== Status ==
Current time: 2022-12-19 12:26:58 (running for 00:02:56.67)
Memory usage on this node: 4.4/23.5 GiB 
Using AsyncHyperBand: num_stopped=14
Bracket: Iter 160.000: -0.5240097127854824 | Iter 80.000: -0.5704203397035599 | Iter 40.000: -0.5757669743150473 | Iter 20.000: -0.5778531371382996 | Iter 10.000: -0.5823181420564651
Resources requested: 0/16 CPUs, 0/0 GPUs, 0.0/12.27 GiB heap, 0.0/6.14 GiB objects
Result logdir: /home/tvangraft/ray_results/run_one_training_2022-12-19_12-24-01
Number of trials: 16/16 (16 TERMINATED)
+------------------------------+------------+---------------------+--------------+-----------------+------------------------+------------------------+----------------+-------------------+----------------+----------+----------------------+
| Trial name                   | status     | loc                 |   batch_size |   learning_rate |   model_attention_head |   model_embedding_size |   model_layers |   scheduler_gamma |   sgd_momentum |     loss |   traini

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
run_one_training_0d7a4_00000,2022-12-19_12-27-37,True,,d8988e71b9a746d6b64fa36d99e66fcf,"0_batch_size=4,learning_rate=0.0500,model_attention_heads=1,model_embedding_size=64,model_layers=1,scheduler_gamma=1,sgd_momentum=0.8000",VGI-DEKSPTOP-TIJMEn,193,0.817341,172.28.206.68,17390,34.9025,0.00335383,34.9025,1671449257,0,,193,0d7a4_00000,0.153913
run_one_training_0d7a4_00001,2022-12-19_12-27-16,True,,e2fdbd63d694463289c54b71bb7a653b,,VGI-DEKSPTOP-TIJMEn,10,0.627226,172.28.206.68,17690,9.71644,0.666147,9.71644,1671449236,0,,10,0d7a4_00001,0.304735
run_one_training_0d7a4_00002,2022-12-19_12-27-27,True,,639d947b97d34be1b37998ad6e3810b2,,VGI-DEKSPTOP-TIJMEn,20,0.613046,172.28.206.68,17692,20.0537,0.734643,20.0537,1671449247,0,,20,0d7a4_00002,0.239057
run_one_training_0d7a4_00003,2022-12-19_12-27-17,True,,6f37bce0ec4349519fdbb4c9b4c2a567,,VGI-DEKSPTOP-TIJMEn,10,0.701942,172.28.206.68,17700,10.5813,0.655334,10.5813,1671449237,0,,10,0d7a4_00003,0.242506
run_one_training_0d7a4_00004,2022-12-19_12-27-17,True,,10a6e400bf0e483b95a7c1234ca05bb8,,VGI-DEKSPTOP-TIJMEn,20,0.597815,172.28.206.68,17706,11.006,0.315164,11.006,1671449237,0,,20,0d7a4_00004,0.296244
run_one_training_0d7a4_00005,2022-12-19_12-27-11,True,,4f1ddd0ef42e4671a64c4acfb7b417e1,,VGI-DEKSPTOP-TIJMEn,10,0.632737,172.28.206.68,17711,4.50424,0.334767,4.50424,1671449231,0,,10,0d7a4_00005,0.261331
run_one_training_0d7a4_00006,2022-12-19_12-27-45,True,,c7b84a37621a4a2aa2c75cfc465de6e1,"6_batch_size=8,learning_rate=0.0500,model_attention_heads=2,model_embedding_size=16,model_layers=3,scheduler_gamma=0.9950,sgd_momentum=0.5000",VGI-DEKSPTOP-TIJMEn,218,0.820624,172.28.206.68,17717,38.6373,0.00316095,38.6373,1671449265,0,,218,0d7a4_00006,0.273391
run_one_training_0d7a4_00007,2022-12-19_12-27-19,True,,1726fb575f8a45998147799d6a74be2d,,VGI-DEKSPTOP-TIJMEn,20,0.612555,172.28.206.68,17723,12.9778,0.398597,12.9778,1671449239,0,,20,0d7a4_00007,0.292318
run_one_training_0d7a4_00008,2022-12-19_12-27-14,True,,c51ab879aa2b45c1b3923784cc5473d6,,VGI-DEKSPTOP-TIJMEn,10,0.663759,172.28.206.68,17729,7.14359,0.587588,7.14359,1671449234,0,,10,0d7a4_00008,0.288217
run_one_training_0d7a4_00009,2022-12-19_12-27-14,True,,81c9da2244e84f41a3c07bb0f471eb1f,,VGI-DEKSPTOP-TIJMEn,10,0.617465,172.28.206.68,17736,7.47614,0.530666,7.47614,1671449234,0,,10,0d7a4_00009,0.293528


== Status ==
Current time: 2022-12-19 12:27:05 (running for 00:00:06.11)
Memory usage on this node: 7.6/23.5 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 160.000: None | Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: -0.6154014126342886
Resources requested: 16.0/16 CPUs, 0/0 GPUs, 0.0/12.27 GiB heap, 0.0/6.14 GiB objects
Result logdir: /home/tvangraft/ray_results/run_one_training_2022-12-19_12-26-59
Number of trials: 16/16 (16 RUNNING)
+------------------------------+----------+---------------------+--------------+-----------------+------------------------+------------------------+----------------+-------------------+----------------+----------+----------------------+
| Trial name                   | status   | loc                 |   batch_size |   learning_rate |   model_attention_head |   model_embedding_size |   model_layers |   scheduler_gamma |   sgd_momentum |     loss |   training_iteration |
|                              |          |       

...

== Status ==
Current time: 2022-12-19 12:34:21 (running for 00:00:06.46)
Memory usage on this node: 7.8/23.5 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 160.000: None | Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: None
Resources requested: 16.0/16 CPUs, 0/0 GPUs, 0.0/12.27 GiB heap, 0.0/6.14 GiB objects
Result logdir: /home/tvangraft/ray_results/run_one_training_2022-12-19_12-34-14
Number of trials: 16/16 (16 RUNNING)
+------------------------------+----------+--------------------+--------------+-----------------+------------------------+------------------------+----------------+-------------------+----------------+----------+----------------------+
| Trial name                   | status   | loc                |   batch_size |   learning_rate |   model_attention_head |   model_embedding_size |   model_layers |   scheduler_gamma |   sgd_momentum |     loss |   training_iteration |
|                              |          |                    |   

2022-12-19 12:37:15,908	INFO tune.py:762 -- Total run time: 181.09 seconds (180.96 seconds for the tuning loop).


== Status ==
Current time: 2022-12-19 12:37:15 (running for 00:03:00.96)
Memory usage on this node: 4.6/23.5 GiB 
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 160.000: -0.5805948767246623 | Iter 80.000: -0.5778572028785041 | Iter 40.000: -0.5832723571973688 | Iter 20.000: -0.5874351333169376 | Iter 10.000: -0.5947970283382079
Resources requested: 0/16 CPUs, 0/0 GPUs, 0.0/12.27 GiB heap, 0.0/6.14 GiB objects
Result logdir: /home/tvangraft/ray_results/run_one_training_2022-12-19_12-34-14
Number of trials: 16/16 (16 TERMINATED)
+------------------------------+------------+--------------------+--------------+-----------------+------------------------+------------------------+----------------+-------------------+----------------+----------+----------------------+
| Trial name                   | status     | loc                |   batch_size |   learning_rate |   model_attention_head |   model_embedding_size |   model_layers |   scheduler_gamma |   sgd_momentum |     loss |   training_

Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
run_one_training_7d84e_00000,2022-12-19_12-40-06,False,,b8f6676706bd4215bede80b55fa334a5,VGI-DEKSPTOP-TIJMEn,137,0.653752,172.28.206.68,14269,166.746,0.9703,166.746,1671450006,0,,137,7d84e_00000,0.16563
run_one_training_7d84e_00001,2022-12-19_12-37-34,True,,c23a4ceb1e8a4d0cabc99eb3c93c05af,VGI-DEKSPTOP-TIJMEn,10,0.686499,172.28.206.68,14544,10.1131,0.929,10.1131,1671449854,0,,10,7d84e_00001,0.275516
run_one_training_7d84e_00002,2022-12-19_12-39-21,True,,659d06a689eb4be8beaf85f280d2913f,VGI-DEKSPTOP-TIJMEn,160,0.673702,172.28.206.68,14549,116.473,0.557829,116.473,1671449961,0,,160,7d84e_00002,0.338809
run_one_training_7d84e_00003,2022-12-19_12-40-06,False,,031a20c0a5d24c92b7f146e47ef84ac8,VGI-DEKSPTOP-TIJMEn,69,0.667776,172.28.206.68,14554,161.652,1.65498,161.652,1671450006,0,,69,7d84e_00003,0.331214
run_one_training_7d84e_00004,2022-12-19_12-38-35,True,,bc2af0c262dc406b8a19b64d30a6ea75,VGI-DEKSPTOP-TIJMEn,20,0.681503,172.28.206.68,14559,70.7658,3.00928,70.7658,1671449915,0,,20,7d84e_00004,0.287825
run_one_training_7d84e_00005,2022-12-19_12-37-41,True,,288d8b8f719f4a1c8fe78a8be99a50d4,VGI-DEKSPTOP-TIJMEn,10,0.689623,172.28.206.68,14565,16.8292,1.45389,16.8292,1671449861,0,,10,7d84e_00005,0.351484
run_one_training_7d84e_00006,2022-12-19_12-38-21,True,,ac0ab0e9e75d4e22986eb35a75b27ef1,VGI-DEKSPTOP-TIJMEn,40,0.683078,172.28.206.68,14570,57.2124,1.29161,57.2124,1671449901,0,,40,7d84e_00006,0.31913
run_one_training_7d84e_00007,2022-12-19_12-38-58,True,,7344aa9e6dd244e39659aad4fcee4761,VGI-DEKSPTOP-TIJMEn,80,0.675004,172.28.206.68,14577,93.542,0.956363,93.542,1671449938,0,,80,7d84e_00007,0.31839
run_one_training_7d84e_00008,2022-12-19_12-40-04,False,,907d369b7d6646db923a3e4866add840,VGI-DEKSPTOP-TIJMEn,172,0.666608,172.28.206.68,14583,159.168,0.774611,159.168,1671450004,0,,172,7d84e_00008,0.322461
run_one_training_7d84e_00009,2022-12-19_12-37-35,True,,f6258b905cbf42498308f951a4d51034,VGI-DEKSPTOP-TIJMEn,10,0.720129,172.28.206.68,14589,10.2674,0.908359,10.2674,1671449855,0,,10,7d84e_00009,0.323949


== Status ==
Current time: 2022-12-19 12:37:23 (running for 00:00:06.61)
Memory usage on this node: 7.9/23.5 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 160.000: None | Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: None
Resources requested: 16.0/16 CPUs, 0/0 GPUs, 0.0/12.27 GiB heap, 0.0/6.14 GiB objects
Result logdir: /home/tvangraft/ray_results/run_one_training_2022-12-19_12-37-17
Number of trials: 16/16 (16 RUNNING)
+------------------------------+----------+---------------------+--------------+-----------------+------------------------+------------------------+----------------+-------------------+----------------+----------+----------------------+
| Trial name                   | status   | loc                 |   batch_size |   learning_rate |   model_attention_head |   model_embedding_size |   model_layers |   scheduler_gamma |   sgd_momentum |     loss |   training_iteration |
|                              |          |                     |